In [1]:
import re
from pyspark.sql import SparkSession, DataFrame, Row
from pyspark.sql import types as T
from pyspark.errors import AnalysisException

from rich.pretty import pprint
from dotenv import load_dotenv

from mlsgpt.dbv2 import store, models, schema
load_dotenv("../.env-deploy", override=True)

True

In [2]:
reader = store.DataReader()

In [3]:
data_home = "/Users/kwesi/Desktop/ai/gpts/mlsgpt/data"
jar_files = ["postgresql-42.7.3.jar", "mysql-connector-j-8.0.33.jar"]
jar_opts = ",".join([f"{data_home}/jars/{jar}" for jar in jar_files])
warehouse = f"{data_home}/warehouse"

spark: SparkSession = (
    SparkSession.builder\
    .appName("MLSGPT")
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.shuffle.service.enabled", "true")
    .config("spark.sql.warehouse.dir", f"{warehouse}")
    .config("spark.sql.session.timeZone", "UTC")
    .config("spark.jars", f"{jar_opts}") 
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")

24/05/20 13:08:10 WARN Utils: Your hostname, marley.local resolves to a loopback address: 127.0.0.1; using 10.0.0.135 instead (on interface en0)
24/05/20 13:08:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/05/20 13:08:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
props = [models.Property.model_validate(prop) for prop in reader.get_properties()]

In [15]:
import tiktoken
output_tokens=174
input_tokens=2586
total_tokens=2760
input_cost = 5.0/1_000_000
output_cost = 15.0/1_000_000
# enc = tiktoken.encoding_for_model("gpt-4")
# tokens = []
# costs = [cost_per_1k_tokens * (token_count / 1000) for token_count in tokens]
total_tokens = input_tokens + output_tokens
total_cost = (input_tokens*input_cost + output_tokens*output_cost)
cost_per_1k_properties = 1000 * total_cost
print(f"Total tokens: {total_tokens}")
print(f"Total cost: ${total_cost:.4f}")
print(f"Cost per 1k properties: ${cost_per_1k_properties:.4f}")

Total tokens: 2760
Total cost: $0.0155
Cost per 1k properties: $15.5400


In [6]:
system_prompt = open("summary_system_prompt.md").read()
prompt_template = "Description JSON:\n```json\n{}\n```"

In [7]:
def parse_text(text: str) -> str:
    extracted = ""
    match = re.search(r'text\n(.*?)', text, re.DOTALL) 
    if match: 
        extracted = match.group(1) 
        return extracted

In [10]:
# from openai import OpenAI
# client = OpenAI()

# def summarize(prop:models.Property) -> str:
#     client = OpenAI()
#     messages = [
#         {"role": "system", "content": system_prompt},
#         {"role": "user", "content": prompt_template.format(prop.model_dump_json())}
#     ]
#     response = client.chat.completions.create(messages=messages, model="gpt-4o")
#     return response
    #description = parse_text(response["choices"][0]["message"]["content"])
    #return Row(ListingID=prop.ListingID, Summary=description, Embedding=None), response

In [11]:
# prop = models.Property.model_validate(props[0])
# response = summarize(prop)

In [12]:
# pprint(response)

ChatCompletion(
│   id='chatcmpl-9R0oSYuqigpuRTS5RiwiaUHYvBWfm',
│   choices=[
│   │   Choice(
│   │   │   finish_reason='stop',
│   │   │   index=0,
│   │   │   logprobs=None,
│   │   │   message=ChatCompletionMessage(
│   │   │   │   content='Welcome to your dream home in Clear Skies, Ilderton! This stunning 3-bedroom, 3-bathroom detached home at 3 Basil Crescent is priced at $1,130,000. Built in 2023 by Sifton Homes, the Black Alder Traditional model offers 2,138 sq. ft. of contemporary living space. The main floor features a chic kitchen with a walk-in pantry, seamlessly connecting to the great room and dining area. Upstairs, the primary bedroom boasts a luxurious ensuite, alongside two more bedrooms, an upper-level laundry closet, and a spacious open loft. Nestled in a quiet area with playgrounds, schools, and shopping nearby, this property offers suburban tranquility with quick access to London. Enjoy four parking spaces and modern amenities like central air conditioning. Seize this opportunity to blend quality and convenience in one perfect home.',
│   │   │   │   role='assistant',
│   │   │   │   function_call=None,
│   │   │   │   tool_calls=None
│   │   │   )
│   │   )
│   ],
│   created=1716225008,
│   model='gpt-4o-2024-05-13',
│   object='chat.completion',
│   system_fingerprint='fp_729ea513f7',
│   usage=CompletionUsage(completion_tokens=174, prompt_tokens=2586, total_tokens=2760)
)

In [ ]:
# import concurrent.futures

# # Define a function to process a batch of embeddings
# def process_batch(batch):
#     with concurrent.futures.ThreadPoolExecutor(100) as executor:
#         futures = []
#         for data in batch:
#             futures.append(executor.submit(summarize, data))

#         results = []
#         for future in concurrent.futures.as_completed(futures):
#             results.append(future.result())
#     return results

# # Split the 'to_embed' list into batches of 1000
# batch_size = 100
# size = len(props)
# batches = [props[i:i+batch_size] for i in range(0, size, batch_size)]

# _schema = T.StructType(
#     [
#         T.StructField("ListingID", T.StringType(), False),
#         T.StructField("Summary", T.TextType(), False),
#         T.StructField("Embedding", T.StringType(), False)
#     ]
# )

# print(f"Processing {len(props)} records in {len(batches)} batches of {batch_size} records each")
# total_processed = 0
# for i, batch in enumerate(batches):
#     if i +  1 > 275:
#         rows = process_batch(batch)
#         df = spark.createDataFrame(rows, _schema)
#         df.write.csv(f"{data_home}/summaries/batch{str(i+1).zfill(6)}.csv", mode="overwrite", header=True)
#         total_processed += len(batch)
#         print(f"Processed batch {i+1} of {len(batches)} for ({total_processed} of {size}) records")

